<a href="https://colab.research.google.com/github/panashematsaudza/Pyspark-Titanic-Logistic-Regression-/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('titanic classification').getOrCreate()

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
dataset = spark.sql('SELECT * FROM titanic_csv')

In [0]:
dataset.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [0]:
dataset.columns

Out[8]: ['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [0]:
my_cols = dataset.select([
  'Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Cabin',
 'Embarked'])

In [0]:
my_final_data = my_cols.na.drop()

In [0]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

In [0]:
gender_indexer = StringIndexer(inputCol= 'Sex' ,outputCol ='SexIndex')
gender_encoder = OneHotEncoder(inputCol ='SexIndex' ,outputCol = 'SexVec')

In [0]:
embark_indexer = StringIndexer(inputCol ='Embarked' ,outputCol = 'EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol = 'EmbarkIndex' ,outputCol = 'EmbarkVec')

In [0]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [0]:
assembler

Out[32]: VectorAssembler_3dde43cfeace

In [0]:
log_reg = LogisticRegression(featuresCol='features',labelCol='Survived')

In [0]:
from pyspark.ml import Pipeline

In [0]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg])

In [0]:
pipeline

Out[36]: Pipeline_8f40adfaac46

In [0]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])

In [0]:
fit_model = pipeline.fit(train_titanic_data)

In [0]:
results = fit_model.transform(test_titanic_data)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [0]:
results.select('Survived','prediction').show()

+--------+----------+
Survived|prediction|
+--------+----------+
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 1.0|
+--------+----------+
only showing top 20 rows

In [0]:
AUC = my_eval.evaluate(results)

In [0]:
AUC


Out[44]: 0.6833333333333335